In [ ]:
import os
import torch
import sklearn.model_selection
from torch import nn
import numpy as np

In [ ]:
from mlxtend.data import mnist_data
X, y = mnist_data()


keep=(y==0) | (y==1)
X=X[keep,:]
y=y[keep]

Xtrain,Xtest,ytrain,ytest=sklearn.model_selection.train_test_split(X,y,test_size=0.3,shuffle=True)


trainDataset=torch.utils.data.TensorDataset(torch.tensor(Xtrain).float(),torch.tensor(ytrain).float())
testDataset=torch.utils.data.TensorDataset(torch.tensor(Xtest).float(),torch.tensor(ytest).float())


In [ ]:
class logRegression(nn.Module):
    def __init__(self):
        super(logRegression, self).__init__()
        self.inputLayer=nn.Linear(784, 1, bias=True)

    def forward(self,x):
      x=self.inputLayer(x)
      x=torch.sigmoid(x)

      return torch.squeeze(x)

testData=torch.rand((20,784))
testNet=logRegression()
testNet.forward(testData)

tensor([0.6139, 0.6129, 0.5125, 0.5988, 0.5469, 0.6298, 0.5160, 0.5630, 0.5711,
        0.5835, 0.5004, 0.5742, 0.5672, 0.5354, 0.5228, 0.5715, 0.5580, 0.5678,
        0.5784, 0.5489], grad_fn=<SqueezeBackward0>)

In [ ]:
class logRegression(nn.Module):
    def __init__(self):
        super(logRegression, self).__init__()
        self.inputLayer=nn.Linear(784, 1, bias=True)

    def forward(self,x):
      x=self.inputLayer(x)
      m=nn.Softmax(dim=1)
      x= m(x)

      return torch.squeeze(x)

testData=torch.rand((20,784))
testNet=logRegression()
testNet.forward(testData)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.], grad_fn=<SqueezeBackward0>)

In [ ]:
nEpoch=10
learningRate=0.001
batchSize=64

trainDataLoader=torch.utils.data.DataLoader(trainDataset,batch_size=batchSize,shuffle=True)

model=logRegression()

optimizer = torch.optim.SGD(model.parameters(), lr=learningRate)

loss_fn=nn.BCELoss()

accuracy=np.zeros(nEpoch)
for iEpoch in range(nEpoch):
  estimatedLabels=np.empty((0,))
  trueLabels=np.empty((0,))
  for xbatch,ybatch in trainDataLoader:
    y_pred=model(xbatch)

    loss = loss_fn(y_pred,ybatch)

    model.zero_grad()

    loss.backward()

    optimizer.step()


    yhat=y_pred>.5
    estimatedLabels=np.append(estimatedLabels,yhat)
    trueLabels=np.append(trueLabels,ybatch)
  accuracy[iEpoch]=np.mean(estimatedLabels==trueLabels)

accuracy

array([0.50142857, 0.50142857, 0.50142857, 0.50142857, 0.50142857,
       0.50142857, 0.50142857, 0.50142857, 0.50142857, 0.50142857])

In [ ]:
nEpoch=10
learningRate=0.001
batchSize=64

trainDataLoader=torch.utils.data.DataLoader(trainDataset,batch_size=batchSize,shuffle=True)

model=nn.Sequential(
    nn.Linear(784, 128),
    nn.ReLU(),
    nn.Linear(128,10),
    nn.Softmax(dim=1)
  )

optimizer = torch.optim.Adam(model.parameters(), lr=learningRate)

# loss_fn=nn.BCELoss()
loss_fn=nn.NLLLoss()

accuracy=np.zeros(nEpoch)
for iEpoch in range(nEpoch):
  estimatedLabels=np.empty((0,))
  trueLabels=np.empty((0,))
  for xbatch,ybatch in trainDataLoader:
    y_pred=model(xbatch)

    # ybatch = nn.functional.one_hot(ybatch.long(), num_classes= 10)

    ybatch= ybatch.long()

    loss = loss_fn(y_pred,ybatch)

    model.zero_grad()

    loss.backward()

    optimizer.step()


    yhat= y_pred.argmax(1)
    estimatedLabels=np.append(estimatedLabels,yhat)
    trueLabels=np.append(trueLabels,ybatch)
  accuracy[iEpoch]=np.mean(estimatedLabels==trueLabels)

accuracy

array([0.61285714, 0.98142857, 0.99142857, 0.99      , 0.98857143,
       0.98857143, 0.99      , 0.99142857, 0.99142857, 0.99571429])

In [ ]:
#test performance:
testDataLoader=torch.utils.data.DataLoader(testDataset,batch_size=batchSize,shuffle=True)

estimatedLabels=np.empty((0,))
trueLabels=np.empty((0,))
for xbatch,ybatch in testDataLoader:
    y_pred=model(xbatch)


    # yhat=y_pred>.5
    yhat= y_pred.argmax(1)
    estimatedLabels=np.append(estimatedLabels,yhat)
    trueLabels=np.append(trueLabels,ybatch)

testAcc=np.mean(estimatedLabels==trueLabels)
print(testAcc)

1.0
